# Huffman encoding

In [1]:
from collections import defaultdict
from math import ceil, log2
import heapq

## Implementation

In [2]:
def calc_freq_chars(path):
    probs = defaultdict(int)
    with open(path, 'r') as text:
        for idx, line in enumerate(text):
            for i in range(len(line)):
                probs[line[i]] += 1

    return probs

In [3]:
text_path = "lab6_lab7/norm_wiki_sample.txt"

In [4]:
calc_freq_chars(text_path)

defaultdict(int,
            {' ': 1840507,
             'a': 777876,
             'l': 378211,
             'b': 145172,
             'e': 1009158,
             'r': 586088,
             't': 715266,
             'o': 627012,
             'f': 190077,
             'p': 184242,
             'u': 229915,
             's': 572689,
             'i': 657640,
             '1': 63329,
             '7': 16523,
             'm': 232270,
             'y': 134244,
             '4': 17341,
             '9': 38410,
             '0': 50436,
             '2': 37553,
             'c': 297462,
             'h': 393431,
             '5': 17809,
             '6': 16484,
             '8': 20745,
             'w': 138676,
             'g': 175671,
             'n': 643628,
             'd': 341036,
             'k': 65072,
             'v': 92206,
             'z': 13933,
             'x': 17630,
             'j': 22956,
             'q': 9205,
             '3': 19038})

Node class will be used for creating the Huffman tree, that is needed for creating the code

In [5]:
class Node:
    def __init__(self, symbol: str, lchild: "Node" = None, rchild: "Node" = None):
        self.symbol = symbol
        self.lchild = lchild
        self.rchild = rchild
        self.code = None

In [6]:
class HuffmanCoding:
    def __init__(self):
        self.nodes = []
        self.root = None
        self.sym_to_code = None
        self.code_to_sym = None

    def add_codes(self, node: Node, parent_code: str) -> None:
        if node.lchild == None: # this means that the node is a leaf -> a single char
            self.sym_to_code[node.symbol] = parent_code
        else:
            self.add_codes(node.lchild, parent_code+'0')
            self.add_codes(node.rchild, parent_code+'1')

    def create(self, char_freq: dict) -> None:
        pq = []
        for char, freq in char_freq.items():
            heapq.heappush(pq, (freq, char))

        while len(pq) > 1:
            fv, v = heapq.heappop(pq)
            fu, u = heapq.heappop(pq)

            n = v+u
            fn = fv + fu

            heapq.heappush(pq, (fn, n))

            new_node = Node(n, Node(v), Node(u))
            
            for node in self.nodes:
                if node.symbol == v:
                    new_node.lchild = node
                elif node.symbol == u:
                    new_node.rchild = node

            self.nodes.append(new_node)
            self.root = new_node

        self.sym_to_code = dict()
        self.add_codes(self.root, '')

        self.code_to_sym = dict()
        for symbol, code in self.sym_to_code.items():
            self.code_to_sym[code] = symbol

    def encode(self, text: str) -> str:
        pos = 0
        cipher = []
        while pos < len(text):
            cipher.append(self.sym_to_code[text[pos]])
            pos += 1

        return "".join(cipher)

    def decode(self, cipher: str) -> str:
        pos = 0
        msg = []
        while pos < len(cipher):
            end_pos = pos+1
            while cipher[pos:end_pos] not in self.code_to_sym:
                end_pos += 1

            msg.append(self.code_to_sym[cipher[pos:end_pos]])
            pos = end_pos

        return "".join(msg)
            

Let's see if it works, `huff.sym_to_code` should contain dictionary with prefix code, were frequently used characters have shorter codes

In [7]:
huff = HuffmanCoding()
huff.create(calc_freq_chars(text_path))
huff.sym_to_code

{'e': '000',
 'm': '00100',
 'y': '001010',
 'k': '0010110',
 '4': '001011100',
 'x': '001011101',
 '5': '001011110',
 '3': '001011111',
 's': '0011',
 'w': '010000',
 'b': '010001',
 'c': '01001',
 'r': '0101',
 'o': '0110',
 'n': '0111',
 'i': '1000',
 'd': '10010',
 '2': '10011000',
 '9': '10011001',
 'v': '1001101',
 'g': '100111',
 't': '1010',
 'p': '101100',
 'f': '101101',
 'l': '10111',
 'a': '1100',
 'h': '11010',
 '8': '110110000',
 'j': '110110001',
 '0': '11011001',
 'q': '1101101000',
 'z': '1101101001',
 '6': '1101101010',
 '7': '1101101011',
 '1': '11011011',
 'u': '110111',
 ' ': '111'}

Let's try encoding and decoding a fragment of text

In [8]:
text_list = []
with open(text_path, 'r') as f:
    text = f.read()

text[:100]

' albert of prussia 17 may 1490 20 march 1568 was the last grand master of the teutonic knights who a'

In [9]:
cipher = huff.encode(text[:100])
cipher

'111110010111010001000010110101110110101101111101100010111011100110011100011001111101101111011010111110010011000010101111101101100101110010011001110110011111001100011011001111001001100010101001110101111101101100101111011011010101101100001110100001100001111110101101000011110111110000111010111100111010111000111100101110010011000011101000001011110110101101111101011010000111101000011011110100110011110000100111100101100111100010011111010101000111110100001101001101111100'

In [10]:
huff.decode(cipher)

' albert of prussia 17 may 1490 20 march 1568 was the last grand master of the teutonic knights who a'

## Conclusions

Now I will compare how the Huffman code is doing compared to the best possible fixed-length encoding.

### Number of bits of text encoded with Huffman encoding

In [11]:
huff_enc = len(huff.encode(text))
huff_enc

46489714

### The shortest possible length of codeword of fixed-length encoding for the text

In [12]:
unique_characters = len(huff.sym_to_code)
fix_len = ceil(log2(unique_characters))
fix_len

6

### Number of bits of text encoded with the best possible fixed-length encoding

In [13]:
fix_len_enc = fix_len * len(text)
fix_len_enc

64733646

### Huffman encoding is ~28% shorter

In [14]:
huff_enc / fix_len_enc

0.718169250037299